In [30]:
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.patches as mpatches
from statistics import mode
from math import sqrt
from fancyimpute import KNN
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


ImportError: cannot import name 'numpy_interface' from partially initialized module 'cvxpy.interface' (most likely due to a circular import) (C:\Users\Epicm\Desktop\data_bases\descriptive_stats_exploratory_anlysis\.venv\lib\site-packages\cvxpy\interface\__init__.py)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
### Reading in our Dataset ###

In [4]:
dataset_df = pd.read_csv(r"C:\Users\Epicm\Desktop\data\knn\knn-training-data.txt")

In [5]:
# let's get rid of customer ID, we have an index
dataset_df = dataset_df.drop('customer_id', axis = 1)

# now let's also get rid of most popular category, a categorical var 
# we don't need in our dataset
dataset_df = dataset_df.drop('most_popular_category', axis = 1)

In [6]:
dataset_df.head()

,relationship_days,total_spend,beverage_categories,segment
0,240,230.75,9,explorer
1,218,117.50,3,explorer
2,139,105.50,4,explorer
3,748,18.00,1,cheapo
4,743,404.50,6,loyal core


In [ ]:
### Creating Training Dataset ###

In [7]:
training_df = dataset_df

#create labels
cols = ['explorer', 'cheapo', 'loyal core', 'newbie']

#add labels
training_df.rename(columns = {cols[0]:0, cols[1]:1, cols[2]:2, cols[3]:3 })

#add a distance column
training_df['distance'] = '9999'

#view our training DF
training_df.head()

,relationship_days,total_spend,beverage_categories,segment,distance
0,240,230.75,9,explorer,9999
1,218,117.50,3,explorer,9999
2,139,105.50,4,explorer,9999
3,748,18.00,1,cheapo,9999
4,743,404.50,6,loyal core,9999


In [ ]:
# Now let's create our target for our KNN, this will be the first row of our training_df.

In [8]:
target = pd.Series([36,240,230.75,9])
#target

In [ ]:
# Now that we have our target we will measure the 'distance' between this target point and all the other points in our dataset. I will be using Euclidean Distance.

In [9]:
training_df['distance'] = (((training_df.loc[:,'relationship_days']-target[0])**2 + (training_df.loc[:,'total_spend']-target[1])**2 + (training_df.loc[:,'beverage_categories']-target[2])**2)** 0.5)


In [10]:
training_df.sort_values('distance', ascending = True).head()

,relationship_days,total_spend,beverage_categories,segment,distance
15725,103,231.25,12,explorer,228.947865
7301,109,237.50,13,explorer,229.674362
1342,108,261.25,12,explorer,231.272837
2877,101,219.00,9,explorer,232.032460
12017,116,228.50,12,explorer,233.203372


In [ ]:
### Building KNN Model ###

In [11]:
k = 10
training_df = training_df.sort_values('distance', ascending = True)
knn = list(training_df.head(k).segment)
knn

['explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer']

In [ ]:
### Reading in Testing Data ###

In [12]:
testing_df = pd.read_csv(r"C:\Users\Epicm\Desktop\data\knn\knn-testing-data.txt")

In [13]:
del testing_df['most_popular_category']

#create a testing prediction column
testing_df['prediction']=''

In [14]:
#choose our target customer to compare the rest too

the_cust = testing_df.iloc[0,:]

In [16]:
# calculate my euclidean distance

training_df['distance'] = (
    ((training_df.loc[:, 'relationship_days'] - the_cust.iloc[1])**2 + 
     (training_df.loc[:, 'total_spend'] - the_cust.iloc[2])**2 + 
     (training_df.loc[:, 'beverage_categories'] - the_cust.iloc[3])**2) ** 0.5
)

## This code calculates the Euclidean distance between each customer in the training_df dataset and a specific customer (the_cust) based on three features: relationship_days, total_spend, and beverage_categories. The distances are stored in a new column called distance. This distance helps quantify how similar each customer is to the specific customer, where smaller values indicate higher similarity.



In [ ]:
## RESULTS ###

In [17]:
training_df.sort_values('distance', ascending=True).head()

,relationship_days,total_spend,beverage_categories,segment,distance
321,224,144.00,4,explorer,1.414214
2407,224,146.00,2,explorer,3.162278
1823,223,148.00,5,explorer,3.162278
5667,227,140.75,6,explorer,5.297405
11612,222,149.50,7,explorer,5.315073


In [18]:
k = 10
training_df = training_df.sort_values('distance', ascending = True)
knn = list(training_df.head(k).segment)
knn

['explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer',
 'explorer']

In [ ]:
## TESTING THE PREDICTION ON ONE DATA SET ROW ###

In [19]:
k = 10

prediction = mode(training_df.sort_values('distance', ascending=True).head(k).segment)

In [20]:
testing_df.iloc[0,4] = prediction

In [21]:
testing_df.head()

,customer_id,relationship_days,total_spend,beverage_categories,prediction
0,", 9GT0P908DS5VGJYRYG4CHAPSQG",224,145.0,5,explorer
1,", D74N0TBT7D2K6YPYDA734V56Y4",272,152.5,5,
2,", XHN902RSGS7V6ZBWN7FE5QVGA4",371,55.0,3,
3,", YDZ8kOpLlCLDlmom",609,159.0,6,
4,-uNXHhkiA5JEebdj,802,107.0,8,


In [ ]:
### Testing Prediction on all dataset Rows ###

In [23]:
k = 25

# This allows us to iterate over all rows in our testing_df
for row_idx in range(testing_df.shape[0]):
    
    # The customer is changed every iteration
    the_cust = testing_df.iloc[row_idx, :]
    
    # Distance calculation with .iloc for proper positional indexing
    training_df['distance'] = (
        ((training_df['relationship_days'] - the_cust.iloc[1]) ** 2 +
         (training_df['total_spend'] - the_cust.iloc[2]) ** 2 +
         (training_df['beverage_categories'] - the_cust.iloc[3]) ** 2) ** 0.5
    )
    
    # Prediction line
    prediction = mode(training_df.sort_values('distance', ascending=True).head(k)['segment'])
    
    # Update the prediction in the testing_df at the corresponding row
    testing_df.iloc[row_idx, 4] = prediction

    # Uncomment this block if you want to limit iterations for debugging
    # if row_idx > 10:
    #     break


In [ ]:
# RESULTS # 

In [24]:
testing_df.value_counts('prediction')

prediction
newbie        562
loyal core    516
cheapo        451
explorer      260
Name: count, dtype: int64

In [25]:
testing_df.head(10)

,customer_id,relationship_days,total_spend,beverage_categories,prediction
0,", 9GT0P908DS5VGJYRYG4CHAPSQG",224,145.00,5,explorer
1,", D74N0TBT7D2K6YPYDA734V56Y4",272,152.50,5,explorer
2,", XHN902RSGS7V6ZBWN7FE5QVGA4",371,55.00,3,cheapo
3,", YDZ8kOpLlCLDlmom",609,159.00,6,loyal core
4,-uNXHhkiA5JEebdj,802,107.00,8,loyal core
5,00GN9QQ53H639C9EW0KJRKGN5W,213,92.00,2,newbie
6,00M2PX68JH0E2Y44BSXPB8WPR8,590,123.00,12,loyal core
7,00PYAN31NH58CYD297MVV2JQTC,559,94.50,5,cheapo
8,00R2NCKF4X5SCSTW59HRY5VDG8,314,79.00,6,newbie
9,00ZSCPVYFH2TEMWQTV6M8PC3K0,393,378.25,13,loyal core


In [ ]:
### Read in Comparison Data ### 

In [26]:
evaluation_df = pd.read_csv(r"C:\Users\Epicm\Desktop\data\knn\testing-true-values.txt")

In [ ]:
### Joining my evaluation_df to my testing_df ###

In [27]:
testing_df = testing_df.merge(evaluation_df, on = 'customer_id')

In [28]:
testing_df.head()

,customer_id,relationship_days,total_spend,beverage_categories,prediction,segment
0,", 9GT0P908DS5VGJYRYG4CHAPSQG",224,145.0,5,explorer,explorer
1,", D74N0TBT7D2K6YPYDA734V56Y4",272,152.5,5,explorer,explorer
2,", XHN902RSGS7V6ZBWN7FE5QVGA4",371,55.0,3,cheapo,cheapo
3,", YDZ8kOpLlCLDlmom",609,159.0,6,loyal core,loyal core
4,-uNXHhkiA5JEebdj,802,107.0,8,loyal core,loyal core


In [ ]:
## Final Results ##

In [32]:
print(confusion_matrix(testing_df.prediction, testing_df.segment))

[[444   0   7   0]
 [  0 258   1   1]
 [  0   3 513   0]
 [  0   1   0 561]]


In [34]:
print(classification_report(testing_df['prediction'], testing_df['segment']))

              precision    recall  f1-score   support

      cheapo       1.00      0.98      0.99       451
    explorer       0.98      0.99      0.99       260
  loyal core       0.98      0.99      0.99       516
      newbie       1.00      1.00      1.00       562

    accuracy                           0.99      1789
   macro avg       0.99      0.99      0.99      1789
weighted avg       0.99      0.99      0.99      1789



In [ ]:
# Here we can see the accuracy of my KNN model is .99.